In [ ]:
!pip install geopy
!pip install category_encoders

In [2]:
import pandas as pd
import geopy.distance
event_data = pd.read_csv("meetup_event_sample.csv")
group_data = pd.read_csv("meetup_group_sample.csv")

radius = 100

event_data.columns,group_data.columns
len(event_data),len(group_data)

total_events = len(event_data)

In [82]:
# Drop unwanted columns from group_data and select relevant columns from event_data

temp1 = group_data.drop(['Unnamed: 0','name'], axis=1)
temp2 = event_data[['group_id', 'venue_lat', 'venue_lon']]

temp3 = pd.merge(temp1, temp2, on='group_id')



df = temp3.drop('group_id',axis=1)


df['category_name'] = df['category_name'].astype('category')

df['category_code'] = df['category_name'].cat.codes


unique_pairs = df.drop_duplicates(subset=['category_name', 'category_code'])

sorted_df = unique_pairs.sort_values(by='category_code')

sorted_df = sorted_df.drop(['venue_lat', 'venue_lon'],axis=1)


df = df.drop('category_name',axis=1)

# shuffled_df = df.sample(frac=1).head(1400)

my_dict = sorted_df.set_index('category_code')['category_name'].to_dict()


# shuffled_df

,venue_lat,venue_lon,category_code
6225,40.755283,-73.982475,24
6799,40.739250,-74.006264,15
5365,40.735153,-73.974648,23
6370,40.744843,-73.983330,24
6769,40.756485,-73.992508,14
...,...,...,...
9537,40.752853,-73.985962,24
9327,40.752865,-73.985992,24
2712,41.194195,-74.184494,19
5495,40.804726,-73.952087,2


***Convert dataframe into numpy to make multiplications easy***

In [85]:
import numpy as np

num = df.to_numpy()


In [84]:
import numpy as np
import geopy.distance

def calc(category, merged_data, radius=1000):
  # Initialize variables
  N = 0
  total_N = len(merged_data)
  repeated_total = np.zeros(30)
  arr = np.zeros(30)
  main_arr = np.zeros(30)
  fin_arr = np.zeros(30)

  # Loop through merged_data to calculate distances and counts
  for i in merged_data:
    # Update repeated_total array with count of each category
    repeated_total[int(i[2])] += 1
    if i[2] == category:
      # If category matches category, calculate distances and counts
      for j in merged_data:
        e1 = (i[0], i[1])
        e2 = (j[0], j[1])
        dist = geopy.distance.geodesic(e1, e2).m 
        if dist < radius:
          N += 1
          arr[int(j[2])] += 1

      # Calculate main_arr
      for k in range(len(arr)):
        val = arr[category] / (N - arr[k])
        main_arr[k] += val

  # Calculate fin_arr
  for i in range(30):
    if repeated_total[i] != 0:
      fin_arr[i] = (total_N - repeated_total[category])/(repeated_total[category] * repeated_total[i])

  # Calculate scores_arr
  scores_arr = np.zeros(30)
  for i in range(30):
    scores_arr[i] = main_arr[i] * fin_arr[i] 

  return scores_arr


In [41]:
all_scores = []

for i in range(30):
  ## for every value get the attrativeness of the corrosponding values 
  all_scores.append(calc(i,num))

all_scores

<ipython-input-40-7bf853b82771>:29: RuntimeWarning: divide by zero encountered in double_scalars
  val = arr[category] / (N - arr[k])
<ipython-input-40-7bf853b82771>:35: RuntimeWarning: divide by zero encountered in double_scalars
  fin_arr[i] = (total_N - repeated_total[category])/(repeated_total[category] * repeated_total[i])
<ipython-input-40-7bf853b82771>:40: RuntimeWarning: invalid value encountered in double_scalars
  scores_arr[i] = main_arr[i] * fin_arr[i]


[array([ 0.83716559,  4.18348581,  0.44355665, 18.73924587, 18.73839477,
         1.73137432,  1.40456334,  5.37708766,  4.15840954,  0.64341247,
         1.72135826,  0.35983077, 18.70959112,  9.37031106,  0.55671315,
         2.37350865,  2.51959626,  0.72061308,  0.47424376,  0.99137583,
         6.25988475, 37.41918224,  0.79889746,  0.64452167,  0.13403881,
         1.39555664,  4.17986162,  0.28855883, 12.486102  ,  0.        ]),
 array([ 0.79227945,  4.03052058,  0.42463123, 17.67456008, 17.70179142,
         1.61109711,  1.36329247,  5.04181174,  3.97095645,  0.58168599,
         1.6475822 ,  0.3510672 , 17.63127791,  8.84960251,  0.50363946,
         2.21361911,  2.36415502,  0.66580521,  0.41897073,  0.94434098,
         5.90463839, 35.26255581,  0.74390062,  0.60372135,  0.11060958,
         1.36760503,  3.9498403 ,  0.31527106, 11.80805355,  0.        ]),
 array([  2.50640108,  12.58646541,   1.38556213,  56.38917889,
         56.41720815,   5.24178168,   4.25655413,  16.17

In [59]:
array_of_arrays = np.vstack(all_scores)


In [60]:
array_of_arrays

array([[8.37165594e-01, 4.18348581e+00, 4.43556645e-01, 1.87392459e+01,
        1.87383948e+01, 1.73137432e+00, 1.40456334e+00, 5.37708766e+00,
        4.15840954e+00, 6.43412474e-01, 1.72135826e+00, 3.59830768e-01,
        1.87095911e+01, 9.37031106e+00, 5.56713147e-01, 2.37350865e+00,
        2.51959626e+00, 7.20613083e-01, 4.74243759e-01, 9.91375832e-01,
        6.25988475e+00, 3.74191822e+01, 7.98897456e-01, 6.44521675e-01,
        1.34038814e-01, 1.39555664e+00, 4.17986162e+00, 2.88558834e-01,
        1.24861020e+01, 0.00000000e+00],
       [7.92279453e-01, 4.03052058e+00, 4.24631228e-01, 1.76745601e+01,
        1.77017914e+01, 1.61109711e+00, 1.36329247e+00, 5.04181174e+00,
        3.97095645e+00, 5.81685988e-01, 1.64758220e+00, 3.51067195e-01,
        1.76312779e+01, 8.84960251e+00, 5.03639460e-01, 2.21361911e+00,
        2.36415502e+00, 6.65805208e-01, 4.18970733e-01, 9.44340981e-01,
        5.90463839e+00, 3.52625558e+01, 7.43900623e-01, 6.03721351e-01,
        1.10609579e-01,

In [64]:
df = pd.DataFrame(array_of_arrays)
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.837166,4.183486,0.443557,18.739246,18.738395,1.731374,1.404563,5.377088,4.158410,0.643412,...,6.259885,37.419182,0.798897,0.644522,0.134039,1.395557,4.179862,0.288559,12.486102,0.0
1,0.792279,4.030521,0.424631,17.674560,17.701791,1.611097,1.363292,5.041812,3.970956,0.581686,...,5.904638,35.262556,0.743901,0.603721,0.110610,1.367605,3.949840,0.315271,11.808054,0.0
2,2.506401,12.586465,1.385562,56.389179,56.417208,5.241782,4.256554,16.177539,12.527381,1.942163,...,18.852242,112.663686,2.408340,1.943317,0.393559,4.188219,12.628056,0.872310,37.602609,0.0
3,7.697144,39.326455,3.978979,351.712025,176.955047,16.086822,13.108818,50.564604,39.321803,5.711269,...,58.987342,353.896226,7.375202,5.806157,0.880900,13.107268,39.323344,2.462907,117.965409,0.0
4,0.487515,2.473911,0.287736,10.900764,11.113279,0.991847,0.824637,3.122754,2.872698,0.353844,...,3.707634,21.782563,0.455402,0.398533,0.071115,0.806762,2.467495,0.155692,7.273545,0.0
5,1.159111,5.844202,0.624845,26.247385,26.248464,2.480484,1.963722,7.542494,5.832058,0.917649,...,8.781685,52.488521,1.111370,0.906361,0.190157,1.952137,5.877764,0.401714,17.496174,0.0
6,0.823404,4.138601,0.457084,18.504321,18.506294,1.703733,1.408500,5.297875,4.113373,0.638361,...,6.192519,36.992705,0.794419,0.635092,0.133376,1.371776,4.129836,0.285660,12.352324,0.0
7,0.210222,1.055737,0.110951,4.745961,4.745961,0.441518,0.354272,1.364558,1.053309,0.164866,...,1.584018,9.479777,0.201822,0.163510,0.035072,0.352268,1.055737,0.073017,3.159926,0.0
8,7.901889,40.278749,4.258350,175.476772,181.254371,15.952434,13.370482,50.136221,53.017693,5.698730,...,58.731047,350.953544,7.679840,6.243665,1.142021,12.998279,40.556934,2.453620,116.984515,0.0
9,2.310163,11.586371,1.236775,52.024992,52.008039,4.847534,3.905166,14.956204,11.553520,1.818430,...,17.356367,103.975392,2.213739,1.794846,0.383404,3.859084,11.596363,0.798245,34.659903,0.0


In [66]:
# df = df.drop(df.columns[-1], axis=1)

# Drop the last row
df = df.drop(df.index[-1], axis=0)
df

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.837166,4.183486,0.443557,18.739246,18.738395,1.731374,1.404563,5.377088,4.158410,0.643412,...,0.991376,6.259885,37.419182,0.798897,0.644522,0.134039,1.395557,4.179862,0.288559,12.486102
1,0.792279,4.030521,0.424631,17.674560,17.701791,1.611097,1.363292,5.041812,3.970956,0.581686,...,0.944341,5.904638,35.262556,0.743901,0.603721,0.110610,1.367605,3.949840,0.315271,11.808054
2,2.506401,12.586465,1.385562,56.389179,56.417208,5.241782,4.256554,16.177539,12.527381,1.942163,...,2.986292,18.852242,112.663686,2.408340,1.943317,0.393559,4.188219,12.628056,0.872310,37.602609
3,7.697144,39.326455,3.978979,351.712025,176.955047,16.086822,13.108818,50.564604,39.321803,5.711269,...,9.313424,58.987342,353.896226,7.375202,5.806157,0.880900,13.107268,39.323344,2.462907,117.965409
4,0.487515,2.473911,0.287736,10.900764,11.113279,0.991847,0.824637,3.122754,2.872698,0.353844,...,0.573724,3.707634,21.782563,0.455402,0.398533,0.071115,0.806762,2.467495,0.155692,7.273545
5,1.159111,5.844202,0.624845,26.247385,26.248464,2.480484,1.963722,7.542494,5.832058,0.917649,...,1.385677,8.781685,52.488521,1.111370,0.906361,0.190157,1.952137,5.877764,0.401714,17.496174
6,0.823404,4.138601,0.457084,18.504321,18.506294,1.703733,1.408500,5.297875,4.113373,0.638361,...,0.982766,6.192519,36.992705,0.794419,0.635092,0.133376,1.371776,4.129836,0.285660,12.352324
7,0.210222,1.055737,0.110951,4.745961,4.745961,0.441518,0.354272,1.364558,1.053309,0.164866,...,0.249468,1.584018,9.479777,0.201822,0.163510,0.035072,0.352268,1.055737,0.073017,3.159926
8,7.901889,40.278749,4.258350,175.476772,181.254371,15.952434,13.370482,50.136221,53.017693,5.698730,...,9.235620,58.731047,350.953544,7.679840,6.243665,1.142021,12.998279,40.556934,2.453620,116.984515
9,2.310163,11.586371,1.236775,52.024992,52.008039,4.847534,3.905166,14.956204,11.553520,1.818430,...,2.748903,17.356367,103.975392,2.213739,1.794846,0.383404,3.859084,11.596363,0.798245,34.659903


In [67]:
df = df.rename(columns=my_dict)


In [71]:
new_col = pd.Series(my_dict)


In [75]:
my_dict

{0: 'Arts & Culture',
 1: 'Book Clubs',
 2: 'Career & Business',
 3: 'Cars & Motorcycles',
 4: 'Community & Environment',
 5: 'Dancing',
 6: 'Education & Learning',
 7: 'Fashion & Beauty',
 8: 'Fitness',
 9: 'Food & Drink',
 10: 'Games',
 11: 'Health & Wellbeing',
 12: 'Hobbies & Crafts',
 13: 'LGBT',
 14: 'Language & Ethnic Identity',
 15: 'Movements & Politics',
 16: 'Movies & Film',
 17: 'Music',
 18: 'New Age & Spirituality',
 19: 'Outdoors & Adventure',
 20: 'Pets & Animals',
 21: 'Photography',
 22: 'Religion & Beliefs',
 23: 'Singles',
 24: 'Socializing',
 25: 'Sports & Recreation',
 26: 'Support',
 27: 'Tech',
 28: 'Writing'}

In [73]:
df.insert(loc=0, column='new_col', value=new_col)

In [74]:
df

,new_col,Arts & Culture,Book Clubs,Career & Business,Cars & Motorcycles,Community & Environment,Dancing,Education & Learning,Fashion & Beauty,Fitness,...,Outdoors & Adventure,Pets & Animals,Photography,Religion & Beliefs,Singles,Socializing,Sports & Recreation,Support,Tech,Writing
0,Arts & Culture,0.837166,4.183486,0.443557,18.739246,18.738395,1.731374,1.404563,5.377088,4.158410,...,0.991376,6.259885,37.419182,0.798897,0.644522,0.134039,1.395557,4.179862,0.288559,12.486102
1,Book Clubs,0.792279,4.030521,0.424631,17.674560,17.701791,1.611097,1.363292,5.041812,3.970956,...,0.944341,5.904638,35.262556,0.743901,0.603721,0.110610,1.367605,3.949840,0.315271,11.808054
2,Career & Business,2.506401,12.586465,1.385562,56.389179,56.417208,5.241782,4.256554,16.177539,12.527381,...,2.986292,18.852242,112.663686,2.408340,1.943317,0.393559,4.188219,12.628056,0.872310,37.602609
3,Cars & Motorcycles,7.697144,39.326455,3.978979,351.712025,176.955047,16.086822,13.108818,50.564604,39.321803,...,9.313424,58.987342,353.896226,7.375202,5.806157,0.880900,13.107268,39.323344,2.462907,117.965409
4,Community & Environment,0.487515,2.473911,0.287736,10.900764,11.113279,0.991847,0.824637,3.122754,2.872698,...,0.573724,3.707634,21.782563,0.455402,0.398533,0.071115,0.806762,2.467495,0.155692,7.273545
5,Dancing,1.159111,5.844202,0.624845,26.247385,26.248464,2.480484,1.963722,7.542494,5.832058,...,1.385677,8.781685,52.488521,1.111370,0.906361,0.190157,1.952137,5.877764,0.401714,17.496174
6,Education & Learning,0.823404,4.138601,0.457084,18.504321,18.506294,1.703733,1.408500,5.297875,4.113373,...,0.982766,6.192519,36.992705,0.794419,0.635092,0.133376,1.371776,4.129836,0.285660,12.352324
7,Fashion & Beauty,0.210222,1.055737,0.110951,4.745961,4.745961,0.441518,0.354272,1.364558,1.053309,...,0.249468,1.584018,9.479777,0.201822,0.163510,0.035072,0.352268,1.055737,0.073017,3.159926
8,Fitness,7.901889,40.278749,4.258350,175.476772,181.254371,15.952434,13.370482,50.136221,53.017693,...,9.235620,58.731047,350.953544,7.679840,6.243665,1.142021,12.998279,40.556934,2.453620,116.984515
9,Food & Drink,2.310163,11.586371,1.236775,52.024992,52.008039,4.847534,3.905166,14.956204,11.553520,...,2.748903,17.356367,103.975392,2.213739,1.794846,0.383404,3.859084,11.596363,0.798245,34.659903
